<div style="display: flex; text-align: centre;justify-contents: centre; align-items: centre;">
 <h1> ML Hackathon </h1> </div>

## Welcome to the  MTech  ML1  (Sem1 ) Hackathon !



## Dataset Description :
Housing price dataset of Bengaluru city is provided as train dataset (train.csv). Along with the train dataset, two more external reference datasets (avg_rent.csv & dist_from_city_centre.csv ) is given for further feature engineering. 
 
Below are the datasets and features details
 
 
### train.csv :
>  * area_type: The type of the house area feature 'total_sqft' specifies.
>  * availability: The availability date or availability status of the property.
>  * location: The locality of the property in Bengaluru city.
>  * size: The size of the housing property in BHK (or Bedrooms etc.,).
>  * society: The name of the Apartment. This name is encrypted for confidentiality.
>  * total_sqft: The 'area_type' area of the property.
>  * bath: Number of bathrooms available in the house.
>  * balcony: Number of balcony/balconies the house has.
>  * price: Price of the housing property in Lakhs. (target feature)
 
The `test.csv` dataset contains similar information to train.csv but does not disclose the “price” feature. The price has to be predicted through your model.

Details of Additional dataset

### avg_rent.csv :
* location : The locality of the property in the bengaluru city.
* avg_2bhk_rent : Average rent of two BHK flat in that location
 
 
### dist_from_city_centre.csv : 
* location : The locality of the property in the bengaluru city.
* dist_from_city : Distance of the location from city center

## Goal :
The objective of this hackathon is to show case your  ML1  learning  particularly  feature engineering skills leveraging primary and secondary datasets to develop a regression model for predicting the house price in Bengaluru. This predictive model will help the  to make purchasing/renting decisions by predicting fair housing prices.

### Metric to measure :
The measure of accuracy will be RMSE (root mean square error). The predicted Price for each house in the test dataset will be compared with the actual Price to calculate the RMSE value of the entire prediction. The lower the RMSE value, the better the model will be.

#### Import the packages and  read data set

In [1]:
import pyforest

In [2]:
house_prices = pd.read_csv("../data-sets/hackathon/train.csv")

house_prices.head()

<IPython.core.display.Javascript object>

,ID,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


#### Data Summary Analysis

In [3]:
house_prices.describe()

,ID,bath,balcony,price
count,10656.000000,10591.000000,10152.000000,10656.000000
mean,5327.500000,2.690586,1.581757,112.214256
std,3076.266568,1.352403,0.821357,142.231967
min,0.000000,1.000000,0.000000,8.000000
25%,2663.750000,2.000000,1.000000,50.000000
50%,5327.500000,2.000000,2.000000,72.000000
75%,7991.250000,3.000000,2.000000,120.000000
max,10655.000000,40.000000,3.000000,2700.000000


Observations:
* `bath` Column is having mean as **2.69*** that is closer to 75% quartile value of its data - _Its Defect_.
* We have to remove the `ID` column, as its a Identifier.

In [4]:
house_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10656 entries, 0 to 10655
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            10656 non-null  int64  
 1   area_type     10656 non-null  object 
 2   availability  10656 non-null  object 
 3   location      10655 non-null  object 
 4   size          10642 non-null  object 
 5   society       6228 non-null   object 
 6   total_sqft    10656 non-null  object 
 7   bath          10591 non-null  float64
 8   balcony       10152 non-null  float64
 9   price         10656 non-null  float64
dtypes: float64(3), int64(1), object(6)
memory usage: 832.6+ KB


In [5]:
null_values = [[item, f"{np.round((house_prices[item].isnull().sum() / len(house_prices))*100, 2)}%" ] for item in house_prices.columns]
# null_values = [house_prices.isnull().sum().sort_values(ascending=False), ((house_prices.isnull().sum() / len(house_prices))*100).sort_values(ascending=False)]
null_data = pd.DataFrame(null_values, columns=['Column', "Null Values"])

null_data

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Column,Null Values
0,ID,0.0%
1,area_type,0.0%
2,availability,0.0%
3,location,0.01%
4,size,0.13%
5,society,41.55%
6,total_sqft,0.0%
7,bath,0.61%
8,balcony,4.73%
9,price,0.0%


Observations:
* The Data Sample is having more than **41.55***% of Data is not available - _Another Defect_ 

Conclusion:
* From the Summary Analysis 

In [6]:
house_prices.society.isnull().sum(), len(house_prices)

(4428, 10656)

In [7]:
house_prices['area_type'].value_counts()

area_type
Super built-up  Area    7009
Built-up  Area          1927
Plot  Area              1651
Carpet  Area              69
Name: count, dtype: int64

In [8]:
total = house_prices.isnull().sum().sort_values(ascending=False)
percent = (house_prices.isnull().sum() / len(house_prices))*100
percent = percent.sort_values(ascending=False)

missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

missing_data

<IPython.core.display.Javascript object>

,Total,Percent
society,4428,41.554054
balcony,504,4.729730
bath,65,0.609985
size,14,0.131381
location,1,0.009384
ID,0,0.000000
area_type,0,0.000000
availability,0,0.000000
total_sqft,0,0.000000
price,0,0.000000


In [9]:
house_prices.describe(include=object)

,area_type,availability,location,size,society,total_sqft
count,10656,10656,10655,10642,6228,10656
unique,4,79,1197,30,2364,1923
top,Super built-up Area,Ready To Move,Whitefield,2 BHK,GrrvaGr,1200
freq,7009,8468,433,4146,70,667
